In [1]:
import json
from pathlib import Path

import pandas as pd

from mechcell import ProcessImage

### Process image example and show results in dataframe

In [4]:
# Select file(s) from directory
meta_dir = Path("../../data/hela_experiments_meta")
data_dir = Path("../../data")

# Select one image to process
meta_files = meta_dir.glob("2022-05-31_Image1.json")

# Select all images from one session
# meta_files = meta_dir.glob("2022-05-31_*.json")

# Select all images
# meta_files = meta_dir.glob("*.json")

In [5]:
dfs = []

# Iterate over metadata file
for meta_file in meta_files:
    with open(meta_file) as f:
        metadata = json.load(f)

        # Create instance of the ProcessImage class
        image_processor = ProcessImage(metadata, data_dir)

        # Execute the image processing pipeline and store the results in a dictionary.
        results = image_processor()
        df = pd.DataFrame.from_dict(results)
        dfs.append(df)

# Concatenate all collected DataFrames into a single DataFrame, resetting the index for continuity.
df = pd.concat(dfs, ignore_index=True)
df

,image_file,cell_type,condition,nucleus_id,voxels_nucleus,major_axis_nucleus,intermediate_axis_nucleus,minor_axis_nucleus,sphericity,elongation,...,minor_axis_HP1,r_dist_random_points,r_dist_condensates,p_minimal_distance_random_points,p_minimal_distance_condensates,physical_pixel_size_z,physical_pixel_size_x,physical_pixel_size_y,pixel_size_um2,voxel_size_um3
0,hela_experiment_images/2022-05-31_Image_1_z_st...,HeLa,Non-Confined,1,3052147.0,17.08426,8.995158,10.374995,0.525473,0.274385,...,1.902082,1.328527,4.374750,2.075364,0.907255,0.172917,0.048881,0.048881,0.002389,0.000413
1,hela_experiment_images/2022-05-31_Image_1_z_st...,HeLa,Non-Confined,1,3052147.0,17.08426,8.995158,10.374995,0.525473,0.274385,...,0.864583,4.594906,5.227062,0.392616,0.826851,0.172917,0.048881,0.048881,0.002389,0.000413
2,hela_experiment_images/2022-05-31_Image_1_z_st...,HeLa,Non-Confined,1,3052147.0,17.08426,8.995158,10.374995,0.525473,0.274385,...,1.556249,3.652110,5.008250,1.386783,1.110526,0.172917,0.048881,0.048881,0.002389,0.000413
3,hela_experiment_images/2022-05-31_Image_1_z_st...,HeLa,Non-Confined,1,3052147.0,17.08426,8.995158,10.374995,0.525473,0.274385,...,1.037499,7.009487,5.667624,1.444454,1.460478,0.172917,0.048881,0.048881,0.002389,0.000413
4,hela_experiment_images/2022-05-31_Image_1_z_st...,HeLa,Non-Confined,1,3052147.0,17.08426,8.995158,10.374995,0.525473,0.274385,...,1.037499,2.832928,8.983064,1.170315,0.147387,0.172917,0.048881,0.048881,0.002389,0.000413
5,hela_experiment_images/2022-05-31_Image_1_z_st...,HeLa,Non-Confined,1,3052147.0,17.08426,8.995158,10.374995,0.525473,0.274385,...,1.210416,5.006063,3.902999,1.254045,0.787207,0.172917,0.048881,0.048881,0.002389,0.000413
6,hela_experiment_images/2022-05-31_Image_1_z_st...,HeLa,Non-Confined,1,3052147.0,17.08426,8.995158,10.374995,0.525473,0.274385,...,0.864583,5.562945,4.852401,0.603884,0.885346,0.172917,0.048881,0.048881,0.002389,0.000413


### Update dataframe and column names

In [6]:
# Scale the distances on major and minor axis for condensates
df["Radial distance HP1a condensate scaled"] = df["r_dist_condensates"] / df["major_axis_nucleus"]
df["Minimum peripheral distance HP1a condensate scaled"] = (
    df["p_minimal_distance_condensates"] / df["minor_axis_nucleus"]
)

# Scale the distances on major and minor axis for random points
df["Radial distance random point scaled"] = df["r_dist_random_points"] / df["major_axis_nucleus"]
df["Minimum peripheral distance random point scaled"] = (
    df["p_minimal_distance_random_points"] / df["minor_axis_nucleus"]
)

# Add volumes of HP1a and nucleus
df["Volume HP1a condensate (um3)"] = df["voxels_condensates"] * df["voxel_size_um3"]
df["Volume nucleus (um3)"] = df["voxels_nucleus"] * df["voxel_size_um3"]

In [7]:
df = df.rename(
    columns={
        "image_file": "Image name",
        "cell_type": "Cell line",
        "condition": "Condition",
        "nucleus_id": "Nucleus ID",
        "voxels_nucleus": "Number of voxels nucleus",
        "major_axis_nucleus": "Major axis nucleus (um)",
        "intermediate_axis_nucleus": "Intermediate axis nucleus (um)",
        "minor_axis_nucleus": "Minor axis nucleus (um)",
        "sphericity": "Sphericity nucleus",
        "elongation": "Elongation nucleus",
        "flatness": "Flatness nucleus",
        "coord_inner_nuc": "Number of voxels inner nucleus object",
        "tot_num_condensates": "Number of total HP1a condensates",
        "condensate_ids": "Condensate ID",
        "voxels_condensates": "Number of voxels HP1a condensates",
        "major_axis_HP1": "Major axis HP1a condensate (um)",
        "intermediate_axis_HP1": "Intermediate axis HP1a condensate (um)",
        "minor_axis_HP1": "Minor axis HP1a condensate (um)",
        "r_dist_random_points": "Radial distance random point (um)",
        "r_dist_condensates": "Radial distance HP1a condensate (um)",
        "p_minimal_distance_random_points": "Peripheral distance random point (um)",
        "p_minimal_distance_condensates": "Peripheral distance HP1a condensate (um)",
        "physical_pixel_size_z": "Physical pixel size Z",
        "physical_pixel_size_x": "Physical pixel size X",
        "physical_pixel_size_y": "Physical pixel size Y",
        "pixel_size_um2": "Pixel size (um2)",
        "voxel_size_um3": "Voxel size (um3)",
    }
)

In [8]:
df

,Image name,Cell line,Condition,Nucleus ID,Number of voxels nucleus,Major axis nucleus (um),Intermediate axis nucleus (um),Minor axis nucleus (um),Sphericity nucleus,Elongation nucleus,...,Physical pixel size X,Physical pixel size Y,Pixel size (um2),Voxel size (um3),Radial distance HP1a condensate scaled,Minimum peripheral distance HP1a condensate scaled,Radial distance random point scaled,Minimum peripheral distance random point scaled,Volume HP1a condensate (um3),Volume nucleus (um3)
0,hela_experiment_images/2022-05-31_Image_1_z_st...,HeLa,Non-Confined,1,3052147.0,17.08426,8.995158,10.374995,0.525473,0.274385,...,0.048881,0.048881,0.002389,0.000413,0.256069,0.087446,0.077763,0.200035,1.285346,1261.029737
1,hela_experiment_images/2022-05-31_Image_1_z_st...,HeLa,Non-Confined,1,3052147.0,17.08426,8.995158,10.374995,0.525473,0.274385,...,0.048881,0.048881,0.002389,0.000413,0.305958,0.079697,0.268956,0.037843,0.159067,1261.029737
2,hela_experiment_images/2022-05-31_Image_1_z_st...,HeLa,Non-Confined,1,3052147.0,17.08426,8.995158,10.374995,0.525473,0.274385,...,0.048881,0.048881,0.002389,0.000413,0.293150,0.107039,0.213770,0.133666,0.223107,1261.029737
3,hela_experiment_images/2022-05-31_Image_1_z_st...,HeLa,Non-Confined,1,3052147.0,17.08426,8.995158,10.374995,0.525473,0.274385,...,0.048881,0.048881,0.002389,0.000413,0.331745,0.140769,0.410289,0.139225,0.126014,1261.029737
4,hela_experiment_images/2022-05-31_Image_1_z_st...,HeLa,Non-Confined,1,3052147.0,17.08426,8.995158,10.374995,0.525473,0.274385,...,0.048881,0.048881,0.002389,0.000413,0.525809,0.014206,0.165821,0.112801,0.234676,1261.029737
5,hela_experiment_images/2022-05-31_Image_1_z_st...,HeLa,Non-Confined,1,3052147.0,17.08426,8.995158,10.374995,0.525473,0.274385,...,0.048881,0.048881,0.002389,0.000413,0.228456,0.075875,0.293022,0.120872,0.530499,1261.029737
6,hela_experiment_images/2022-05-31_Image_1_z_st...,HeLa,Non-Confined,1,3052147.0,17.08426,8.995158,10.374995,0.525473,0.274385,...,0.048881,0.048881,0.002389,0.000413,0.284028,0.085335,0.325618,0.058206,0.266489,1261.029737


In [9]:
df.to_csv("../../data/hela_results/df_HeLa_data.csv")